# FOCUSED Project: OSPO adoption

As part of the [FOCUSED Collaboration project](https://github.com/JumpsuitWizard/FOCUSED-Collaboration), this notebook examines OSPO (Open Source Programs Offices) adoption across the [Standard and Poor's 500 index](https://en.wikipedia.org/wiki/S%26P_500).

## Authors

- **PI**: Duane O'Brien
- **Researcher**: julia ferraioli
- **Analyst**: Reshama Shaikh

## Research question

## Methodology

## Data sources

The following data sources are used in the analysis:

- [S&P 500](https://github.com/datasets/s-and-p-500-companies/blob/master/data/constituents.csv) retrieved on 2021-10-05
- [OSCI Index](https://opensourceindex.io/) retrieved on 2022-02-28
- [OSPO Landscape](https://landscape.todogroup.org/) retrieved on 2022-05-09


## Data fetching

Get the latest version of the data if the cached copy is more than 1 month old


In [13]:
import csv
from datetime import date
import json
import numpy as np
import pandas as pd
import plotly.express as px
import pprint
import os
import re
import requests
from yaml import safe_load

pp = pprint.PrettyPrinter(indent=2)

# Set the raw output directory in YYYY/MM format
today = date.today()
data_dir = "data_raw/%s/%s/" % (today.year, ('%02d' % (today.month)))

# Dictionary of sources
sources = {"SP500": {"name": "S&P 500",
                     "link": "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/main/data/constituents.csv",
                     "format": "csv",
                     "data": None
                     },
           "OSCI": {"name": "OSCI",
                    # This is the base URL; needs to be fully qualified with year/month.json (ex /monthly/2023/07.json)
                    "link": "https://ststaticprodosciwebz2vmu.blob.core.windows.net/data/osci-ranking/monthly/%s/%s.json" % (today.year, ('%02d' % (today.month-1))),
                    "format": "json",
                    "data": None
                    },
           "TODO": {"name": "TODO Group Landscape",
                    "link": "https://landscape.todogroup.org/api/export?category=todo-group-member,ospo-adopter&project=&license=&organization=&headquarters=&company-type=&industries=&sort=name&grouping=no&bestpractices=&enduser=&parent=&language=&specification=&format=main",
                    "format": "csv",
                    "data": None
                    },
           "OSPOPlusPlus": {"name": "OSPO++",
                            "link": "https://raw.githubusercontent.com/ospoplusplus/ospoplusplus/main/content/about/members/_index.md",
                            "format": "md",
                            "data": None
                            },
           "OSPOAlliance": {"name": "OSPO Alliance",
                            "link": "https://gitlab.eclipse.org/eclipse/plato/www/-/raw/main/layouts/shortcodes/section-members.html",
                            "format": "html",
                            "data": None
                            }
           }

# Fetch and save the versioned data if it is out of date
for key, value in sources.items():
    # Check most recent data file
    filepath = os.path.join(data_dir, ("%s.%s" % (key, value['format'])))
    if os.path.isfile(filepath):
        print(value['name'], " is up-to-date")
    else:
        # Request the data
        try:
            req = requests.get(value['link'], stream=True)
            req.raise_for_status()
            if not os.path.exists(data_dir):
                os.makedirs(data_dir)
            with open(os.path.join(data_dir, ("%s.%s" % (key, value['format']))), "w") as f:
                f.write(req.text)
        except requests.exceptions.HTTPError as e:
            print("received %s; skipping." % (e))
            continue


S&P 500  is up-to-date
OSCI  is up-to-date
TODO Group Landscape  is up-to-date
OSPO++  is up-to-date
OSPO Alliance  is up-to-date


## Data setup


### Create utility functions for parsing specific data sources


In [14]:
# Process OSPO++ members
def parse_opp(filepath):
    regex = r"(?<=company name\=)\"(.*?)\""
    with open(filepath, 'rt') as f:
        raw = f.read()
        return pd.DataFrame({'OSPO++ Member': re.findall(regex, raw)})

# Process OSPO Alliance members
def parse_alliance(filepath):
    name_re = r"(?<=alt\=)\"(.*?)(?= logo|\")"
    site_re = r"(?<=href\=)\"(.*?)(?=\")"
    with open(filepath, 'r') as f:
        raw = f.read()
        return pd.DataFrame({'OSPO Alliance Member': re.findall(name_re, raw), 'Website': re.findall(site_re, raw)})


### Load the data into Pandas


In [15]:
# Parse and process fetched data files
for key, value in sources.items():
    format = value['format']
    filepath = os.path.join(data_dir, ("%s.%s" % (key, format)))
    if os.path.isfile(filepath) is False:
        continue
    if format == 'csv':
        value['data'] = pd.read_csv(filepath)
    elif format == 'json':
        value['data'] = pd.read_json(filepath)
        value['data'] = pd.json_normalize(value['data']['data'])
    elif format == 'yml' or format == 'yaml':
        print("file %s has no currently implemented handler" % (key))
    elif format == 'md':
        if key == 'OSPOPlusPlus':
            value['data'] = parse_opp(filepath)
        else:
             print("file %s has no currently implemented handler" % (key))
    elif format == 'html':
        if key == 'OSPOAlliance':
            value['data'] = parse_alliance(filepath)
        else:
             print("file %s has no currently implemented handler" % (key))
    else:
        print("file format %s has no currently implemented handler" % (format))
            

### Preview the data


In [16]:
for key, value in sources.items():
    print(key, "\n------")
    pp.pprint(value['data'].sample(5))
    print("\n")

SP500 
------
    Symbol              Security             GICS Sector  \
185   EVRG                Evergy               Utilities   
488    WDC       Western Digital  Information Technology   
329    MOS  Mosaic Company (The)               Materials   
417    NOW            ServiceNow  Information Technology   
63     BDX      Becton Dickinson             Health Care   

                              GICS Sub-Industry       Headquarters Location  \
185                          Electric Utilities       Kansas City, Missouri   
488  Technology Hardware, Storage & Peripherals        San Jose, California   
329        Fertilizers & Agricultural Chemicals              Tampa, Florida   
417                            Systems Software     Santa Clara, California   
63                        Health Care Equipment  Franklin Lakes, New Jersey   

     Date added      CIK             Founded  
185  2018-06-05  1711269                1909  
488  2009-07-01   106040                1970  
329  2011

### Clean up the data


In [17]:
# Rename columns in S&P 500 and add a few fields for comparison purposes
sources['SP500']['data'] = sources['SP500']['data'].rename(
    columns={'Name': 'company', 'Sector': 'sector'})

sources['SP500']['data']['in S&P 500'] = True
sources['SP500']['data']['country'] = "United States"

# Reorder the columns for clarity's sake
order = ['company', 'sector', 'country', 'in S&P 500']

sources['SP500']['data'] = sources['SP500']['data'].reindex(order, axis=1)

# Filter out columns from OSCI that we don't need
keep_cols = ['company', 'position', 'industry']
sources['OSCI']['data'] = sources['OSCI']['data'].filter(keep_cols)
# print(df_osci.head())

# Rename columns in OSCI and add a field for comparison purposes
sources['OSCI']['data'] = sources['OSCI']['data'].rename(
    columns={'position': 'OSCI position', 'industry': 'OSCI sector'})
sources['OSCI']['data']['in OSCI'] = True

# Reorder the columns for clarity's sake
order = ['company', 'OSCI sector', 'OSCI position', 'in OSCI']

sources['OSCI']['data'] = sources['OSCI']['data'].reindex(order, axis=1)

# Rename columns in OSPO Landscape
sources['TODO']['data'] = sources['TODO']['data'].rename(columns={
    'Name': 'TODO status',
    'Organization': 'company',
    'Market Cap': 'market cap',
    'Crunchbase Country': 'TODO country'
})

# Only keep certain columns in the data set
sources['TODO']['data'] = sources['TODO']['data'].filter(
    ['TODO status', 'company', 'market cap', 'TODO country'])

# Add a field for comparison purposes
sources['TODO']['data']['in TODO landscape'] = True

# Filter out those who have not adopted an OSPO
sources['TODO']['data'] = sources['TODO']['data'].loc[sources['TODO']['data']['TODO status'].str.contains(
    "adopter", case=False)]

# Reorder the columns for clarity's sake
order = ['company', 'TODO country', 'market cap',
         'TODO status', 'in TODO landscape']

sources['TODO']['data'] = sources['TODO']['data'].reindex(order, axis=1)


### Preview the data


In [18]:
for key, value in sources.items():
    print(key, "\n------")
    pp.pprint(value['data'].sample(5))
    print("\n")


SP500 
------
     company  sector        country  in S&P 500
97       NaN     NaN  United States        True
0        NaN     NaN  United States        True
491      NaN     NaN  United States        True
485      NaN     NaN  United States        True
60       NaN     NaN  United States        True


OSCI 
------
                   company       OSCI sector  OSCI position  in OSCI
88                ScyllaDB        Technology             89     True
142          Open Robotics        Technology            143     True
112                MariaDB        Technology            113     True
199  Mellanox Technologies  Media & Telecoms            200     True
279             Improbable        Technology            280     True


TODO 
------
            company   TODO country    market cap               TODO status  \
181      Salesforce  United States  1.987216e+11      Salesforce (Adopter)   
164  Morgan Stanley  United States  1.516777e+11  Morgan Stanley (Adopter)   
170           Nokia 

### Merge the data sources


In [643]:
# First, merge S&P Index and OSCI, then merge with OSPO Landscape
all_data = (df_sp.merge(df_osci, left_on='company',
                        right_on='company', how='outer')).merge(df_ospo,
                                                                left_on='company',
                                                                right_on='company',
                                                                how='outer')

# Prefer S&P data over OSCI and OSPO Landscape data
all_data['sector'] = all_data['sector'].mask(
    pd.isnull, all_data['OSCI sector'])
all_data['country'] = all_data['country'].mask(
    pd.isnull, all_data['OSPO country'])
all_data = all_data.drop(['OSCI sector', 'OSPO country'], axis=1)

all_data.sort_values(by=['company']).head(10)


,company,sector,country,in S&P 500,OSCI position,in OSCI,market cap,OSPO status,in OSPO landscape
0,3M,Industrials,United States,True,NaN,NaN,NaN,NaN,NaN
768,4teamwork,Technology,NaN,NaN,285.0,True,NaN,NaN,NaN
759,5minds,Technology,NaN,NaN,276.0,True,NaN,NaN,NaN
1,A. O. Smith,Industrials,United States,True,NaN,NaN,NaN,NaN,NaN
7,ADM,Consumer Staples,United States,True,NaN,NaN,NaN,NaN,NaN
11,AES Corp,Utilities,United States,True,NaN,NaN,NaN,NaN,NaN
526,AMD,Technology,NaN,NaN,31.0,True,NaN,NaN,NaN
44,APA Corporation,Energy,United States,True,NaN,NaN,NaN,NaN,NaN
519,ARM,Technology,NaN,NaN,22.0,True,NaN,NaN,NaN
51,AT&T,Communication Services,United States,True,NaN,NaN,NaN,NaN,NaN


### Do some manual cleanup for known issues


In [644]:
# Google is split across two Alphabet stock options and Google so we'll merge them
googles = all_data.loc[all_data['company'].str.contains(
    'Alphabet|Google', case=False, regex=True)]
col_order = ['company', 'sector', 'country', 'in S&P 500', 'OSCI position', 'in OSCI', 'market cap',
             'OSPO status', 'in OSPO landscape']
google = googles.groupby('country', as_index=False).last()

# Update the data set and drop the extraneous entries
all_data.set_index('company', inplace=True)
all_data.update(google.set_index('company'))
all_data.reset_index(inplace=True)
all_data[all_data['company'] == 'Google']
all_data.drop(googles.iloc[:2].index, inplace=True)


In [645]:
all_data.sample(25)


,company,sector,country,in S&P 500,OSCI position,in OSCI,market cap,OSPO status,in OSPO landscape
87,CarMax,Consumer Discretionary,United States,True,NaN,NaN,NaN,NaN,NaN
47,Aptiv,Consumer Discretionary,United States,True,NaN,NaN,NaN,NaN,NaN
601,Truss Works,Technology,NaN,NaN,116.0,True,NaN,NaN,NaN
412,Sherwin-Williams,Materials,United States,True,NaN,NaN,NaN,NaN,NaN
405,Salesforce,Information Technology,United States,True,30.0,True,1.747803e+11,Salesforce (Adopter),True
642,ArangoDB,Technology,NaN,NaN,158.0,True,NaN,NaN,NaN
65,Best Buy,Consumer Discretionary,United States,True,NaN,NaN,NaN,NaN,NaN
411,ServiceNow,Information Technology,United States,True,NaN,NaN,NaN,NaN,NaN
619,Kaltura,Technology,NaN,NaN,135.0,True,NaN,NaN,NaN
200,Ford,Consumer Discretionary,United States,True,NaN,NaN,NaN,NaN,NaN


In [646]:
# Normalize the sectors across data sets

all_data['sector'].mask(all_data['sector'] ==
                        'Information Technology', 'Technology', inplace=True)
all_data['sector'].mask(all_data['sector'] == 'Health Care',
                        'Healthcare & Pharma', inplace=True)
all_data['sector'].mask(all_data['sector'] == 'Financials',
                        'Banking, Insurance & Financial Services ', inplace=True)

# Fill null values with default ones where needed
all_data = all_data.fillna(value={'in S&P 500': False, 'in OSPO landscape': False, 'in OSCI': False,
                                  'OSCI position': 'n/a', 'OSPO status': 'n/a', 'market cap': 'unknown', 'sector': 'unknown'})


### Get a sneak peak at the data


In [647]:
data_sample = all_data.sample(20)
data_sample.sort_values(by=['company'])


,company,sector,country,in S&P 500,OSCI position,in OSCI,market cap,OSPO status,in OSPO landscape
19,Align Technology,Healthcare & Pharma,United States,True,n/a,False,unknown,n/a,False
21,Alliant Energy,Utilities,United States,True,n/a,False,unknown,n/a,False
533,Automattic,Technology,NaN,False,38.0,True,unknown,n/a,False
77,Broadcom,Technology,United States,True,70.0,True,unknown,n/a,False
656,Buro Happold,Professional Services,NaN,False,172.0,True,unknown,n/a,False
88,Carnival Corporation,Consumer Discretionary,United States,True,n/a,False,unknown,n/a,False
105,Chipotle Mexican Grill,Consumer Discretionary,United States,True,n/a,False,unknown,n/a,False
106,Chubb,"Banking, Insurance & Financial Services",United States,True,n/a,False,unknown,n/a,False
172,Entergy,Utilities,United States,True,n/a,False,unknown,n/a,False
177,Essex Property Trust,Real Estate,United States,True,n/a,False,unknown,n/a,False


## Inspect the data


In [648]:
sp_count = len(all_data[all_data['in S&P 500']])
ospo_count = len(all_data[all_data['in OSPO landscape']])
osci_count = len(all_data[all_data['in OSCI']])
sp_ospo_count = len(all_data.query('`in S&P 500` & `in OSPO landscape`'))
sp_osci_count = len(all_data.query('`in S&P 500` & `in OSCI`'))
ospo_osci = len(all_data.query('`in OSPO landscape` & `in OSCI`'))
intersection = len(all_data.query(
    '`in S&P 500` & `in OSPO landscape` & `in OSCI`'))

listings = pd.DataFrame([[sp_count, ospo_count, osci_count, sp_ospo_count, sp_osci_count, ospo_osci, intersection]],
                        ['count'], ['in S&P 500', 'in OSPO landscape', 'in OSCI', 'in S&P and OSPO landscape',
                        'in S&P and OSCI', 'in OSPO landscape and  OSCI', 'in all three'])
listings


,in S&P 500,in OSPO landscape,in OSCI,in S&P and OSPO landscape,in S&P and OSCI,in OSPO landscape and OSCI,in all three
count,504,102,299,23,23,36,12


### What companies are present in all three datasets?


In [649]:
all_data.query('`in S&P 500` & `in OSPO landscape` & `in OSCI`')


,company,sector,country,in S&P 500,OSCI position,in OSCI,market cap,OSPO status,in OSPO landscape
8,Adobe,Technology,United States,True,17.0,True,188740124672.0,Adobe (Adopter),True
45,Apple,Technology,United States,True,24.0,True,2608140386304.0,Apple (Adopter),True
53,Autodesk,Technology,United States,True,98.0,True,41994772480.0,Autodesk (Adopter),True
220,Goldman Sachs,"Banking, Insurance & Financial Services",United States,True,248.0,True,107847974912.0,Goldman Sachs (Adopter),True
228,Hewlett Packard Enterprise,Technology,United States,True,40.0,True,20440801280.0,HPE (Adopter),True
240,IBM,Technology,United States,True,5.0,True,120321998848.0,IBM (Adopter),True
248,Intel,Technology,United States,True,4.0,True,183187193856.0,Intel (Adopter),True
314,Microsoft,Technology,United States,True,2.0,True,2087771176960.0,Microsoft (Adopter),True
328,Netflix,Communication Services,United States,True,77.0,True,85567168512.0,Netflix (Adopter),True
405,Salesforce,Technology,United States,True,30.0,True,174780301312.0,Salesforce (Adopter),True


## Save processed data to file


In [650]:
all_data.to_csv('data_derived/merged_data.csv', index=False)


## Proceed to create visualizations

Head over to the [Visualizations notebook](Visualizations.ipynb) to generate some charts about the data.
